# Coloring of a random graph using Belief Propagation

Given a graph $G=(V,E)$ and an integer $q$, a valid coloring is an assignment mapping each vertex in $G$ to a color (i.e. an integer in $\{1,2,\ldots,q\}$) such that no edge connects two nodes with the same color.
One way to solve the problem is to write a probability distribution over all the possible assignments giving non-zero weights to the valid ones, zero otherwise
$$p(x)\propto \prod_{(ij)\in E}\left[1-\delta(x_i,x_j)\right]$$
which is then approximated via BP.

In [27]:
using Graphs, IndexedGraphs, IndexedFactorGraphs
using BeliefPropagation, BeliefPropagation.Models
using Random
using GraphRecipes, Plots

In [28]:
# try to find a valid coloring using BP
function try_coloring(g::SimpleGraph, q::Integer; 
        maxiter=100, tol=1e-8, damp=0.5, rein=1e-2)
    N = nv(g)
    gp = pairwise_interaction_graph(IndexedGraph(g))
    ψ = fill(ColoringCoupling(), ne(g))
    ϕ = map(1:N) do i
        ϕᵢ = exp.(1e-6*randn(MersenneTwister(seed), q))
        BPFactor(ϕᵢ)
    end
    states = fill(q, N)
    bp = BP(gp, ψ, states; ϕ)
    iterate!(bp; maxiter, tol, damp, rein)
    x = argmax.(beliefs(bp))
    return x
end

# compute the number of monochromatic edges, i.e. those with
#  the same color at both ends
function n_monochromatic(x, g::SimpleGraph)
    en = 0
    for e in edges(g)
        i = src(e); j = dst(e)
        en += x[i] == x[j]
    end
    return en
end

function is_valid_coloring(x, g::SimpleGraph)
    return n_monochromatic(x, g) == 0
end

is_valid_coloring (generic function with 1 method)

In [47]:
q = 3
N = 100
c = 2.5
seed = 1

g = erdos_renyi(N, round(Int, N*c/2); seed)

maxiter = 10^3
tol = 1e-8
damp = 0.5
rein = 1e-2

x = try_coloring(g, q; maxiter, tol, damp, rein)
is_valid_coloring(x, g)

true